In [4]:
!pip install -r requirement.txt

  Using cached matplotlib-3.9.2-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached psutil-6.1.0-cp37-abi3-win_amd64.whl.metadata (23 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached matplotlib-3.9.2-cp311-cp311-win_amd64.whl (7.8 MB)
Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl (12.9 MB)
Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl (11.6 MB)
Using cached psutil-6.1.0-cp37-abi3-win_amd64.whl (254 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
  Attempting uninstall: psutil
    Found existing installation: psutil 5.9.8
    Uninstalling psutil-5.9.8:
      Successfully uninstalled psutil-5.9.8
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found exist

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.17.0 requires numpy<2.0.0,>=1.23.5; python_version <= "3.11", but you have numpy 2.1.3 which is incompatible.
torchvision 0.19.0 requires numpy<2, but you have numpy 2.1.3 which is incompatible.


In [5]:
import subprocess
import serial

import re
import time
import os

import csv
import pandas as pd
import matplotlib.pyplot as plt

import glob

In [1]:
# Folder output utama
code_folders = ["aes128/aes128rfida", "aes128/aes128rfidb", "aes128/aes128rfidc", "aes128/aes128rfidd", "aes128/aes128rfide"]

# Buat folder utama jika belum ada
for folder in code_folders:
    os.makedirs(folder, exist_ok=True)

def generate_code_with_decrementing_plaintext(file_path, id_values):
    # Template kode Arduino
    code_template = f"""
#include <SPI.h>
#include <MFRC522.h>
#include <Wire.h>
#include <Adafruit_INA219.h>
#include <Crypto.h>
#include <AES.h>
#include <MemoryFree.h>
#include <LiquidCrystal_I2C.h>

#define SS_PIN 10
#define RST_PIN 9

MFRC522 rfid(SS_PIN, RST_PIN);
Adafruit_INA219 ina219;
LiquidCrystal_I2C lcd(0x27, 16, 4);

AES128 aes128;
byte buffer[16];

byte ciphertext[{id_values}];
byte decryptedtext[{id_values}];
unsigned long encryptionTime;
unsigned long decryptionTime;

float encryptionVoltage;
float encryptionCurrent;
float decryptionVoltage;
float decryptionCurrent;

const byte keyAES128[16] = {{ 0x0f, 0x0e, 0x0d, 0x0c, 0x0b, 0x0a, 0x09, 0x08, 
                            0x07, 0x06, 0x05, 0x04, 0x03, 0x02, 0x01, 0x00 }};

void encryptAndPrint(BlockCipher *cipher, const byte *key, size_t keySize, byte *plaintext, size_t plaintextSize) {{
  unsigned long startTime = micros();
  cipher->setKey(key, keySize);

  for (size_t i = 0; i < plaintextSize; i += 16) {{
    cipher->encryptBlock(buffer, &plaintext[i]);
    memcpy(&ciphertext[i], buffer, 16);
  }}

  unsigned long endTime = micros();
  encryptionVoltage = ina219.getBusVoltage_V();
  encryptionCurrent = ina219.getCurrent_mA();
  encryptionTime = endTime - startTime;

  // Output encrypted data (ciphertext)
  Serial.print(F("Ciphertext: "));
  for (size_t i = 0; i < plaintextSize; i++) {{
    if (ciphertext[i] < 0x10) Serial.print("0");
    Serial.print(ciphertext[i], HEX);
    Serial.print(" ");
  }}
  Serial.println();
  Serial.print(F("Encryption Completed in "));
  Serial.print(encryptionTime);
  Serial.println(F(" us"));
}}

void decryptAndPrint(BlockCipher *cipher, const byte *key, size_t keySize, byte *ciphertext, size_t ciphertextSize) {{
    unsigned long startdecTime = micros();
    cipher->setKey(key, keySize);

    for (size_t i = 0; i < ciphertextSize; i += 16) {{
        cipher->decryptBlock(buffer, &ciphertext[i]);
        memcpy(&decryptedtext[i], buffer, 16);
    }}

    unsigned long enddecTime = micros();
    decryptionVoltage = ina219.getBusVoltage_V();
    decryptionCurrent = ina219.getCurrent_mA();
    decryptionTime = enddecTime - startdecTime;

    // Output decrypted data
    Serial.print(F("Decrypted Text: "));
    for (size_t i = 0; i < ciphertextSize; i++) {{
        if (decryptedtext[i] < 0x10) Serial.print("0");
        Serial.print(decryptedtext[i], HEX);
        Serial.print(" ");
    }}
    Serial.println();
    Serial.print(F("Decryption Completed in "));
    Serial.print(decryptionTime);
    Serial.println(F(" us"));
}}

void displayData() {{
    float encryptionPower = encryptionVoltage * (encryptionCurrent / 1000);
    float decryptionPower = decryptionVoltage * (decryptionCurrent / 1000);
    int memoryUsage = 2048 - freeMemory();

    Serial.print(F("Memory Usage = "));
    Serial.print(memoryUsage);
    Serial.println(F(" B"));
    Serial.println();

    Serial.print(F("Encryption Voltage: "));
    Serial.print(encryptionVoltage);
    Serial.println(F(" V"));
    Serial.print(F("Encryption Current: "));
    Serial.print(encryptionCurrent);
    Serial.println(F(" mA"));
    Serial.print(F("Encryption Power: "));
    Serial.print(encryptionPower);
    Serial.println(F(" W"));
    Serial.println();
    
    Serial.print(F("Decryption Voltage: "));
    Serial.print(decryptionVoltage);
    Serial.println(F(" V"));
    Serial.print(F("Decryption Current: "));
    Serial.print(decryptionCurrent);
    Serial.println(F(" mA"));
    Serial.print(F("Decryption Power: "));
    Serial.print(decryptionPower);
    Serial.println(F(" W"));

    Serial.println(F("selesai"));
}}

void setup() {{
    Serial.begin(9600);
    SPI.begin();
    rfid.PCD_Init();
    ina219.begin();
    
    lcd.begin(16, 4);
    lcd.backlight();
    lcd.setCursor(0, 0);
    lcd.print("RFID System Ready");
    delay(2000);
    lcd.clear();
}}

void loop() {{
    if (rfid.PICC_IsNewCardPresent() && rfid.PICC_ReadCardSerial()) {{
        byte cardID[{id_values}] = {{0}};
        size_t idLength = rfid.uid.size;

        for (byte i = 0; i < idLength && i < {id_values}; i++) {{
            cardID[i] = rfid.uid.uidByte[i];
        }}

        if (idLength < {id_values}) {{
            memset(&cardID[idLength], 0, {id_values} - idLength);
        }}

        lcd.clear();
        lcd.setCursor(0, 0);
        lcd.print("RFID Detected!");
        lcd.setCursor(0, 1);
        lcd.print("Encrypting...");

        encryptAndPrint(&aes128, keyAES128, sizeof(keyAES128), cardID, {id_values});
        decryptAndPrint(&aes128, keyAES128, sizeof(keyAES128), ciphertext, {id_values});
        displayData();

        lcd.clear();
        lcd.setCursor(0, 0);
        lcd.print("Encryption Done!");
        delay(2000);
        lcd.clear();
    }}
}}
"""

    with open(file_path, 'w') as file:
        file.write(code_template)

    print(f"Code with {id_values}-byte plaintext has been written to {file_path}")

# Generate folder and kode
for folder in code_folders:
    for i in range(304, 15, -16):
        filename = f"aes128rfid_{i}"
        sketch_folder = os.path.join(folder, filename)
        os.makedirs(sketch_folder, exist_ok=True)  # Membuat folder untuk setiap file

        file_path = os.path.join(sketch_folder, f"{filename}.ino")

        generate_code_with_decrementing_plaintext(file_path, i)


Code with 304-byte plaintext has been written to aes128/aes128rfida\aes128rfid_304\aes128rfid_304.ino
Code with 288-byte plaintext has been written to aes128/aes128rfida\aes128rfid_288\aes128rfid_288.ino
Code with 272-byte plaintext has been written to aes128/aes128rfida\aes128rfid_272\aes128rfid_272.ino
Code with 256-byte plaintext has been written to aes128/aes128rfida\aes128rfid_256\aes128rfid_256.ino
Code with 240-byte plaintext has been written to aes128/aes128rfida\aes128rfid_240\aes128rfid_240.ino
Code with 224-byte plaintext has been written to aes128/aes128rfida\aes128rfid_224\aes128rfid_224.ino
Code with 208-byte plaintext has been written to aes128/aes128rfida\aes128rfid_208\aes128rfid_208.ino
Code with 192-byte plaintext has been written to aes128/aes128rfida\aes128rfid_192\aes128rfid_192.ino
Code with 176-byte plaintext has been written to aes128/aes128rfida\aes128rfid_176\aes128rfid_176.ino
Code with 160-byte plaintext has been written to aes128/aes128rfida\aes128rfid_160

In [ ]:
# Konfigurasi
port = "COM5"
baud_rate = 9600
arduino_cli_path = "C:\\Users\\nandi\\Downloads\\arduino-ide_nightly-20241013_Windows_64bit\\resources\\app\\lib\\backend\\resources\\arduino-cli.exe"
log_base_folder = "../../logs"

In [3]:
# a

base_folder = "aes128/aes128rfida"

# Memastikan base log folder ada
os.makedirs(log_base_folder, exist_ok=True)

def compile_and_upload(sketch_path, log_file):
    # Compile dan upload kode ke Arduino
    compile_command = [arduino_cli_path, "compile", "--fqbn", "arduino:avr:nano", sketch_path]
    upload_command = [arduino_cli_path, "upload", "-p", port, "--fqbn", "arduino:avr:nano", sketch_path]

    # Jalankan perintah compile  dan mengambil output dari penggunaan memori
    compile_result = subprocess.run(compile_command, capture_output=True, text=True)
    upload_result = subprocess.run(upload_command, capture_output=True, text=True)
    time.sleep(2) 

    # Ekstrak flash dan penggunaan RAM dari keluaran compile
    flash_usage = ram_usage = None
    flash_match = re.search(r"Sketch uses (\d+) bytes.*Maximum is (\d+) bytes", compile_result.stdout)
    ram_match = re.search(r"Global variables use (\d+) bytes.*Maximum is (\d+) bytes", compile_result.stdout)
    if flash_match and ram_match:
        flash_usage = f"{flash_match.group(1)} / {flash_match.group(2)} bytes"
        ram_usage = f"{ram_match.group(1)} / {ram_match.group(2)} bytes"
        print(f"{os.path.basename(sketch_path)} - Flash: {flash_usage}, RAM: {ram_usage}")
    else:
        print(f"{os.path.basename(sketch_path)} - Failed to retrieve memory usage information")

    # Tulis Flash dan RAM pada log file
    with open(log_file, "w") as f:
        if flash_usage and ram_usage:
            f.write(f"Flash Usage: {flash_usage}, RAM Usage: {ram_usage}\n\n")
        else:
            f.write("Failed to retrieve memory usage information\n\n")

    return compile_result.returncode == 0 and upload_result.returncode == 0

def record_serial_data(log_file):
    with serial.Serial(port, baud_rate, timeout=1) as ser, open(log_file, 'a') as file:
        ser.flushInput()
        start_time = time.time()
        while time.time() - start_time < 10:
            if ser.in_waiting > 0:
                line = ser.readline().decode('utf-8').strip()
                file.write(line + '\n')
                print(line)
                if 'selesai' in line.lower():
                    break
        time.sleep(1)

log_folder = os.path.join(log_base_folder, f"logs_{base_folder}")
os.makedirs(log_folder, exist_ok=True)

subfolders = sorted(
    [folder for folder in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, folder))],
    key=lambda x: int(x.split('_')[1])
)

for folder_name in subfolders:
    sketch_path = os.path.join(base_folder, folder_name, f"{folder_name}.ino")
    if os.path.isfile(sketch_path):
        print(f"Running {folder_name} in {base_folder}...")

        log_file = os.path.join(log_folder, f"{folder_name}.txt")

        if compile_and_upload(sketch_path, log_file):
            record_serial_data(log_file)
        else:
            print(f"{folder_name} failed to run in {base_folder}.\n")
    else:
        print(f"File {folder_name}.ino not found in {folder_name} within {base_folder}")

print("Data collection complete.")


Running aes128rfid_16 in aes128/aes128rfida...
aes128rfid_16.ino - Flash: 16612 / 30720 bytes, RAM: 1021 / 2048 bytes
Ciphertext: 3F 99 60 0F F7 0B A5 DD 0B 78 AC 14 4E 47 FF 34
Encryption Completed in 712 us
Decrypted Text: 23 EE AE 0D 00 00 00 00 00 00 00 00 00 00 00 00
Decryption Completed in 1244 us
Memory Usage = 1058 B

Encryption Voltage: 4.20 V
Encryption Current: 7.00 mA
Encryption Power: 0.03 W

Decryption Voltage: 4.20 V
Decryption Current: 7.20 mA
Decryption Power: 0.03 W
selesai
Running aes128rfid_32 in aes128/aes128rfida...
aes128rfid_32.ino - Flash: 16682 / 30720 bytes, RAM: 1053 / 2048 bytes
Ciphertext: 3F 99 60 0F F7 0B A5 DD 0B 78 AC 14 4E 47 FF 34 E5 31 13 21 91 8C 38 6E 63 E9 8D FF 0A FA 77 0D
Encryption Completed in 1288 us
Decrypted Text: 23 EE AE 0D 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
Decryption Completed in 2332 us
Memory Usage = 1090 B

Encryption Voltage: 4.20 V
Encryption Current: 7.40 mA
Encryption Power: 0.03 

In [ ]:
# b

base_folder = "aes128/aes128rfidb"

os.makedirs(log_base_folder, exist_ok=True)

def compile_and_upload(sketch_path, log_file):
    compile_command = [arduino_cli_path, "compile", "--fqbn", "arduino:avr:nano", sketch_path]
    upload_command = [arduino_cli_path, "upload", "-p", port, "--fqbn", "arduino:avr:nano", sketch_path]

    compile_result = subprocess.run(compile_command, capture_output=True, text=True)
    upload_result = subprocess.run(upload_command, capture_output=True, text=True)
    time.sleep(2)
    
    flash_usage = ram_usage = None
    flash_match = re.search(r"Sketch uses (\d+) bytes.*Maximum is (\d+) bytes", compile_result.stdout)
    ram_match = re.search(r"Global variables use (\d+) bytes.*Maximum is (\d+) bytes", compile_result.stdout)
    if flash_match and ram_match:
        flash_usage = f"{flash_match.group(1)} / {flash_match.group(2)} bytes"
        ram_usage = f"{ram_match.group(1)} / {ram_match.group(2)} bytes"
        print(f"{os.path.basename(sketch_path)} - Flash: {flash_usage}, RAM: {ram_usage}")
    else:
        print(f"{os.path.basename(sketch_path)} - Failed to retrieve memory usage information")

    with open(log_file, "w") as f:
        if flash_usage and ram_usage:
            f.write(f"Flash Usage: {flash_usage}, RAM Usage: {ram_usage}\n\n")
        else:
            f.write("Failed to retrieve memory usage information\n\n")

    return compile_result.returncode == 0 and upload_result.returncode == 0

def record_serial_data(log_file):
    with serial.Serial(port, baud_rate, timeout=1) as ser, open(log_file, 'a') as file:
        start_time = time.time()
        while time.time() - start_time < 10:
            if ser.in_waiting > 0:
                line = ser.readline().decode('utf-8').strip()
                file.write(line + '\n')
                print(line)
                if 'selesai' in line.lower():
                    break
        time.sleep(1)

log_folder = os.path.join(log_base_folder, f"logs_{base_folder}")
os.makedirs(log_folder, exist_ok=True)

subfolders = sorted(
    [folder for folder in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, folder))],
    key=lambda x: int(x.split('_')[1])
)

for folder_name in subfolders:
    sketch_path = os.path.join(base_folder, folder_name, f"{folder_name}.ino")
    if os.path.isfile(sketch_path):
        print(f"Running {folder_name} in {base_folder}...")

        log_file = os.path.join(log_folder, f"{folder_name}.txt")

        if compile_and_upload(sketch_path, log_file):
            record_serial_data(log_file)
        else:
            print(f"{folder_name} failed to run in {base_folder}.\n")
    else:
        print(f"File {folder_name}.ino not found in {folder_name} within {base_folder}")

print("Data collection complete.")


In [ ]:
# c

base_folder = "aes128/aes128rfidc"

os.makedirs(log_base_folder, exist_ok=True)

def compile_and_upload(sketch_path, log_file):
    compile_command = [arduino_cli_path, "compile", "--fqbn", "arduino:avr:nano", sketch_path]
    upload_command = [arduino_cli_path, "upload", "-p", port, "--fqbn", "arduino:avr:nano", sketch_path]

    compile_result = subprocess.run(compile_command, capture_output=True, text=True)
    upload_result = subprocess.run(upload_command, capture_output=True, text=True)
    time.sleep(2)
    
    flash_usage = ram_usage = None
    flash_match = re.search(r"Sketch uses (\d+) bytes.*Maximum is (\d+) bytes", compile_result.stdout)
    ram_match = re.search(r"Global variables use (\d+) bytes.*Maximum is (\d+) bytes", compile_result.stdout)
    if flash_match and ram_match:
        flash_usage = f"{flash_match.group(1)} / {flash_match.group(2)} bytes"
        ram_usage = f"{ram_match.group(1)} / {ram_match.group(2)} bytes"
        print(f"{os.path.basename(sketch_path)} - Flash: {flash_usage}, RAM: {ram_usage}")
    else:
        print(f"{os.path.basename(sketch_path)} - Failed to retrieve memory usage information")

    with open(log_file, "w") as f:
        if flash_usage and ram_usage:
            f.write(f"Flash Usage: {flash_usage}, RAM Usage: {ram_usage}\n\n")
        else:
            f.write("Failed to retrieve memory usage information\n\n")

    return compile_result.returncode == 0 and upload_result.returncode == 0

def record_serial_data(log_file):
    with serial.Serial(port, baud_rate, timeout=1) as ser, open(log_file, 'a') as file:
        start_time = time.time()
        while time.time() - start_time < 10:
            if ser.in_waiting > 0:
                line = ser.readline().decode('utf-8').strip()
                file.write(line + '\n')
                print(line)
                if 'selesai' in line.lower():
                    break
        time.sleep(1)

log_folder = os.path.join(log_base_folder, f"logs_{base_folder}")
os.makedirs(log_folder, exist_ok=True)

subfolders = sorted(
    [folder for folder in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, folder))],
    key=lambda x: int(x.split('_')[1])
)

for folder_name in subfolders:
    sketch_path = os.path.join(base_folder, folder_name, f"{folder_name}.ino")
    if os.path.isfile(sketch_path):
        print(f"Running {folder_name} in {base_folder}...")

        log_file = os.path.join(log_folder, f"{folder_name}.txt")

        if compile_and_upload(sketch_path, log_file):
            record_serial_data(log_file)
        else:
            print(f"{folder_name} failed to run in {base_folder}.\n")
    else:
        print(f"File {folder_name}.ino not found in {folder_name} within {base_folder}")

print("Data collection complete.")


In [ ]:
# d

base_folder = "aes128/aes128rfidd"

os.makedirs(log_base_folder, exist_ok=True)

def compile_and_upload(sketch_path, log_file):
    compile_command = [arduino_cli_path, "compile", "--fqbn", "arduino:avr:nano", sketch_path]
    upload_command = [arduino_cli_path, "upload", "-p", port, "--fqbn", "arduino:avr:nano", sketch_path]

    compile_result = subprocess.run(compile_command, capture_output=True, text=True)
    upload_result = subprocess.run(upload_command, capture_output=True, text=True)
    time.sleep(2)
    
    flash_usage = ram_usage = None
    flash_match = re.search(r"Sketch uses (\d+) bytes.*Maximum is (\d+) bytes", compile_result.stdout)
    ram_match = re.search(r"Global variables use (\d+) bytes.*Maximum is (\d+) bytes", compile_result.stdout)
    if flash_match and ram_match:
        flash_usage = f"{flash_match.group(1)} / {flash_match.group(2)} bytes"
        ram_usage = f"{ram_match.group(1)} / {ram_match.group(2)} bytes"
        print(f"{os.path.basename(sketch_path)} - Flash: {flash_usage}, RAM: {ram_usage}")
    else:
        print(f"{os.path.basename(sketch_path)} - Failed to retrieve memory usage information")

    with open(log_file, "w") as f:
        if flash_usage and ram_usage:
            f.write(f"Flash Usage: {flash_usage}, RAM Usage: {ram_usage}\n\n")
        else:
            f.write("Failed to retrieve memory usage information\n\n")

    return compile_result.returncode == 0 and upload_result.returncode == 0

def record_serial_data(log_file):
    with serial.Serial(port, baud_rate, timeout=1) as ser, open(log_file, 'a') as file:
        start_time = time.time()
        while time.time() - start_time < 10:
            if ser.in_waiting > 0:
                line = ser.readline().decode('utf-8').strip()
                file.write(line + '\n')
                print(line)
                if 'selesai' in line.lower():
                    break
        time.sleep(1)

log_folder = os.path.join(log_base_folder, f"logs_{base_folder}")
os.makedirs(log_folder, exist_ok=True)

subfolders = sorted(
    [folder for folder in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, folder))],
    key=lambda x: int(x.split('_')[1])
)

for folder_name in subfolders:
    sketch_path = os.path.join(base_folder, folder_name, f"{folder_name}.ino")
    if os.path.isfile(sketch_path):
        print(f"Running {folder_name} in {base_folder}...")

        log_file = os.path.join(log_folder, f"{folder_name}.txt")

        if compile_and_upload(sketch_path, log_file):
            record_serial_data(log_file)
        else:
            print(f"{folder_name} failed to run in {base_folder}.\n")
    else:
        print(f"File {folder_name}.ino not found in {folder_name} within {base_folder}")

print("Data collection complete.")


In [ ]:
# e

base_folder = "aes128/aes128rfide"

os.makedirs(log_base_folder, exist_ok=True)

def compile_and_upload(sketch_path, log_file):
    compile_command = [arduino_cli_path, "compile", "--fqbn", "arduino:avr:nano", sketch_path]
    upload_command = [arduino_cli_path, "upload", "-p", port, "--fqbn", "arduino:avr:nano", sketch_path]

    compile_result = subprocess.run(compile_command, capture_output=True, text=True)
    upload_result = subprocess.run(upload_command, capture_output=True, text=True)
    time.sleep(2)
    
    flash_usage = ram_usage = None
    flash_match = re.search(r"Sketch uses (\d+) bytes.*Maximum is (\d+) bytes", compile_result.stdout)
    ram_match = re.search(r"Global variables use (\d+) bytes.*Maximum is (\d+) bytes", compile_result.stdout)
    if flash_match and ram_match:
        flash_usage = f"{flash_match.group(1)} / {flash_match.group(2)} bytes"
        ram_usage = f"{ram_match.group(1)} / {ram_match.group(2)} bytes"
        print(f"{os.path.basename(sketch_path)} - Flash: {flash_usage}, RAM: {ram_usage}")
    else:
        print(f"{os.path.basename(sketch_path)} - Failed to retrieve memory usage information")

    with open(log_file, "w") as f:
        if flash_usage and ram_usage:
            f.write(f"Flash Usage: {flash_usage}, RAM Usage: {ram_usage}\n\n")
        else:
            f.write("Failed to retrieve memory usage information\n\n")

    return compile_result.returncode == 0 and upload_result.returncode == 0

def record_serial_data(log_file):
    with serial.Serial(port, baud_rate, timeout=1) as ser, open(log_file, 'a') as file:
        start_time = time.time()
        while time.time() - start_time < 10:
            if ser.in_waiting > 0:
                line = ser.readline().decode('utf-8').strip()
                file.write(line + '\n')
                print(line)
                if 'selesai' in line.lower():
                    break
        time.sleep(1)

log_folder = os.path.join(log_base_folder, f"logs_{base_folder}")
os.makedirs(log_folder, exist_ok=True)

subfolders = sorted(
    [folder for folder in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, folder))],
    key=lambda x: int(x.split('_')[1])
)

for folder_name in subfolders:
    sketch_path = os.path.join(base_folder, folder_name, f"{folder_name}.ino")
    if os.path.isfile(sketch_path):
        print(f"Running {folder_name} in {base_folder}...")

        log_file = os.path.join(log_folder, f"{folder_name}.txt")

        if compile_and_upload(sketch_path, log_file):
            record_serial_data(log_file)
        else:
            print(f"{folder_name} failed to run in {base_folder}.\n")
    else:
        print(f"File {folder_name}.ino not found in {folder_name} within {base_folder}")

print("Data collection complete.")


### Convert to CSV

In [6]:
log_base_folder = "../../logs/logs_aes128"
output_base_folder = "../../Data_CSV/128/CSVs"

os.makedirs(output_base_folder, exist_ok=True)

header = [
    "file_name", "flash_usage", "ram_usage", "memory_usage",
    "encryption_time", "encryption_voltage", "encryption_current", "encryption_power",
    "decryption_time", "decryption_voltage", "decryption_current", "decryption_power"
]

def parse_txt_file(file_path):
    with open(file_path, 'r') as f:
        data = f.read()

    flash_match = re.search(r"Flash Usage: (\d+) / \d+ bytes", data)
    ram_match = re.search(r"RAM Usage: (\d+) / \d+ bytes", data)
    memory_match = re.search(r"Memory Usage = (\d+) B", data)

    encryption_time_match = re.search(r"Encryption Completed in (\d+) us", data)
    decryption_time_match = re.search(r"Decryption Completed in (\d+) us", data)

    encryption_voltage_match = re.search(r"Encryption Voltage: ([\d.]+) V", data)
    encryption_current_match = re.search(r"Encryption Current: ([\d.]+) mA", data)
    decryption_voltage_match = re.search(r"Decryption Voltage: ([\d.]+) V", data)
    decryption_current_match = re.search(r"Decryption Current: ([\d.]+) mA", data)

    flash_usage = int(flash_match.group(1)) if flash_match else None
    ram_usage = int(ram_match.group(1)) if ram_match else None
    memory_usage = int(memory_match.group(1)) if memory_match else None
    encryption_time = int(encryption_time_match.group(1)) if encryption_time_match else None
    decryption_time = int(decryption_time_match.group(1)) if decryption_time_match else None
    encryption_voltage = float(encryption_voltage_match.group(1)) if encryption_voltage_match else None
    encryption_current = float(encryption_current_match.group(1)) if encryption_current_match else None
    decryption_voltage = float(decryption_voltage_match.group(1)) if decryption_voltage_match else None
    decryption_current = float(decryption_current_match.group(1)) if decryption_current_match else None

    encryption_power = encryption_voltage * (encryption_current / 1000) if encryption_voltage and encryption_current else None
    decryption_power = decryption_voltage * (decryption_current / 1000) if decryption_voltage and decryption_current else None

    return {
        "file_name": os.path.splitext(os.path.basename(file_path))[0],
        "flash_usage": flash_usage,
        "ram_usage": ram_usage,
        "memory_usage": memory_usage,
        "encryption_time": encryption_time,
        "encryption_voltage": encryption_voltage,
        "encryption_current": encryption_current,
        "encryption_power": encryption_power,
        "decryption_time": decryption_time,
        "decryption_voltage": decryption_voltage,
        "decryption_current": decryption_current,
        "decryption_power": decryption_power
    }

for log_folder in os.listdir(log_base_folder):
    log_folder_path = os.path.join(log_base_folder, log_folder)
    if os.path.isdir(log_folder_path) and log_folder.startswith("aes128rfid"):
        output_csv = os.path.join(output_base_folder, f"{log_folder}.csv")

        all_data = []

        txt_files = sorted(
            [file for file in os.listdir(log_folder_path) if file.endswith(".txt")],
            key=lambda x: int(re.search(r'aes128rfid_(\d+)', x).group(1)) 
        )

        for file_name in txt_files:
            file_path = os.path.join(log_folder_path, file_name)
            file_data = parse_txt_file(file_path)
            all_data.append(file_data)

        with open(output_csv, 'w', newline='') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=header)
            writer.writeheader()
            writer.writerows(all_data)

        print(f"Data from {log_folder} has been successfully saved to {output_csv}")


Data from aes128rfida has been successfully saved to ../../Data_CSV/128/CSVs\aes128rfida.csv
Data from aes128rfidb has been successfully saved to ../../Data_CSV/128/CSVs\aes128rfidb.csv
Data from aes128rfidc has been successfully saved to ../../Data_CSV/128/CSVs\aes128rfidc.csv
Data from aes128rfidd has been successfully saved to ../../Data_CSV/128/CSVs\aes128rfidd.csv
Data from aes128rfide has been successfully saved to ../../Data_CSV/128/CSVs\aes128rfide.csv


### Average

In [ ]:
# Tentukan direktori yang berisi file CSV
directory = "../../Data_CSV/128/CSVs"

# Tentukan output path untuk menyimpan hasil rata-rata
output_path = "../../Data_CSV/128/avg_csv/avg_csv.csv"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

file_paths = glob.glob(os.path.join(directory, '*.csv'))

# Periksa apakah ada file CSV yang ditemukan
if not file_paths:
    print("No CSV files found in the specified directory.")
else:
    dfs = [pd.read_csv(file) for file in file_paths]
    
    average_df = dfs[0].copy()

    for col in average_df.select_dtypes(include='number').columns:
        average_df[col] = sum(df[col] for df in dfs) / len(dfs)

    # Tambahkan kolom byte_size berdasarkan jumlah baris
    byte_sizes = list(range(16, 16 * len(average_df) + 1, 16))

    if len(byte_sizes) != len(average_df):
        raise ValueError("Mismatch between number of rows in DataFrame and byte_sizes.")

    average_df.insert(1, 'byte_size', byte_sizes)

    average_df['file_name'] = average_df['byte_size'].apply(lambda x: f"aes128rfid_{x}")

    average_df['sort_key'] = average_df['file_name'].str.extract(r'(\d+)').astype(int)
    sorted_df = average_df.sort_values('sort_key').drop(columns=['sort_key']).reset_index(drop=True)
    
    sorted_df.to_csv(output_path, index=False)
    print(f"Averaged and sorted CSV file saved to: {output_path}")


Averaged and sorted CSV file saved to: ../../Data_CSV/128/avg_csv/avg_csv.csv


Deleting byte_size = 304

In [9]:
file_path = "../../Data_CSV/128/avg_csv/avg_csv.csv"
df = pd.read_csv(file_path)

# Menghapus baris dengan nilai byte_size == 304
df_cleaned = df[df['byte_size'] != 304]

# Menyimpan kembali ke file CSV 
output_path = "../../Data_CSV/128/avg_csv/avg_csv.csv"
df_cleaned.to_csv(output_path, index=False)

print("byte_size 304 has been deleted.")


byte_size 304 has been deleted.
